In [59]:
import os
import re
import glob
import emoji
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string


In [60]:
product_list = pd.read_csv('unique_product_list_with_ratings.csv')
q = product_list['Pages'].quantile(.95)
product_list = product_list[product_list['Pages'] < q]
product_list.reset_index(drop=True, inplace=True)
product_list['Pages'] = product_list['Pages'].astype(dtype='int')
product_list['Total Reviews'] = product_list['Total Reviews'].apply(lambda x: x.replace(',','')).astype(
    dtype='int')
print(len(product_list))
product_list.head(1)


317


,Brand,Name,Price,Link,Features,Overall Rating,Total Reviews,Pages,5,4,3,2,1
0,Asus Mobile,ASUS ZenFone 5Z,"36,299",https://www.flipkart.com/asus-zenfone-5z-midnight-blue-128-gb/p/itmf5g47reyumstg?pid=MOBF5G47CSQGGRCX&lid=LSTMOBF5G47CSQGGRCX0L48BE&marketplace=FLIPKART&store=tyy%2F4io&srno=b_1_1&otracker=browse&fm=organic&iid=76c371fa-106b-41b4-a17e-4a0668322fe0.MOBF5G47CSQGGRCX.SEARCH&ppt=None&ppn=None&ssid=847ns653m80000001675070146926,"['6 GB RAM | 128 GB ROM | Expandable Upto 2 TB', '15.75 cm (6.2 inch) Full HD+ Display', '12MP + 8MP | 8MP Front Camera', '3300 mAh Battery', 'Qualcomm Snapdragon 845 Octa Core Processor', 'Brand Warranty of 1 Year Available for Handset and 6 Months for Accessories']",4.5,4616,462,15005,4566,979,339,1052


In [61]:
product_list['Total Reviews'].sum()

672580

In [62]:
# path = os.getcwd()
# csv_files = glob.glob(os.path.join(path, "reviews/*.csv"))
# data = pd.DataFrame(columns=['product', 'username', 'user_rating', 'title', 'user_review', 'review_posted_time'])
# dfs = []
# for f in csv_files:
#     dfs.append(pd.read_csv(f))

# data = pd.concat([df for df in dfs])
# data.head()

# Preprocessing steps
1. Demojize
2. Lower casing
3. Remove special characters
4. Links / URLs
5. Remove stopwords
6. Converting words to root word (Stemming / Lemmatization)
6. Handle chat conversations*
7. Text Correction
8. 

In [63]:
data = pd.read_csv('reviews_201_to_last.csv')
data.head()


,product,username,user_rating,title,user_review,review_posted_time
0,IQOO 9T 5G,Moin Pops,4.0,Best in the market!,Awesome 👌,2 months ago
1,IQOO 9T 5G,Flipkart Customer,5.0,Worth every penny,So smoothly game performance,3 months ago
2,IQOO 9T 5G,Flipkart Customer,5.0,Great product,Best camera in this segment.. Just I loved it 🔥,30 days ago
3,IQOO 9T 5G,Karthik H,5.0,Terrible product,Low quality mobile not worth even for 20k Feeling like ordinary mobile,1 month ago
4,IQOO 9T 5G,Bhargav Bhargav,1.0,Decent product,According to price ok and specifications and design is the main plus point.,12 days ago


In [64]:
data.drop(['username', 'review_posted_time'], axis=1, inplace=True)


In [65]:
data.shape

(799258, 4)

In [66]:
data.nunique()

product           96
user_rating       15
title           1645
user_review    47134
dtype: int64

In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799258 entries, 0 to 799257
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   product      799258 non-null  object 
 1   user_rating  799258 non-null  float64
 2   title        799247 non-null  object 
 3   user_review  799243 non-null  object 
dtypes: float64(1), object(3)
memory usage: 24.4+ MB


In [68]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
user_rating,799258.0,4.212835,1.131823,1.0,4.0,5.0,5.0,5.0


##### 1. Demojize

In [69]:
data['title'] = data['title'].apply(lambda text: emoji.demojize(str(text)))
data['user_review'] = data['user_review'].apply(
    lambda text: emoji.demojize(str(text)))


##### 2. Lower casing

In [70]:
for col in data.columns[data.dtypes == 'O']:
    data[col] = data[col].str.lower()

#### 1. Links / URLs

In [71]:
# regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"

regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)))"

pattern = re.compile(regex)

data['user_review'] = data['user_review'].apply(
    lambda text: pattern.sub(r'', text))


#### 4.Remove special characters

In [72]:
exclude = string.punctuation
print(exclude)
def remove_punc_(text):
    return text.translate(str.maketrans(exclude, ' '*len(exclude), ''))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [73]:
for col in data.columns[data.dtypes == 'O']:
    data[col] = data[col].apply(remove_punc_)

In [74]:
pd.set_option("max_colwidth", 1000)

In [47]:
data.head(10)

,product,user_rating,title,user_review
0,iqoo 9t 5g,4.0,best in the market,awesome okhand
1,iqoo 9t 5g,5.0,worth every penny,so smoothly game performance
2,iqoo 9t 5g,5.0,great product,best camera in this segment just i loved it fire
3,iqoo 9t 5g,5.0,terrible product,low quality mobile not worth even for 20k feeling like ordinary mobile
4,iqoo 9t 5g,1.0,decent product,according to price ok and specifications and design is the main plus point
5,iqoo 9t 5g,3.0,pretty good,great
6,iqoo z6 5g,4.0,terrific,nice mobile must buy performance is best nice display wonderful camera experience
7,iqoo z6 5g,5.0,mindblowing purchase,this phone is very good and here the phone runs fast and is not laging the battery of this phone is very good and the camera is very good its selfie camera
8,iqoo z6 5g,5.0,wonderful,after 1month used my experience was very goodnyc design all is one excellent phoneso i am giving 5star rb kapoor from silvassa
9,iqoo z6 5g,5.0,pretty good,good product


#### 5. Removing stopwords

Product --> shows related names while typing (can be selected or not)
if selected --> show dashboard
else show dashboard of related product(s)

Rating & Review (OPTIONAL / CONDITIONAL) --> SENTIMENT by category

In [ ]:
FLOW

more the rating ---> more positve